In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [2]:
import geopandas as gpd
import pandas as pd

## Load daily temperature 

In [3]:
years = [year for year in range(1960, 2021)]

In [4]:
pattern = []
for year in years:
    temp_year = ddf.read_csv(r"/global/cfs/cdirs/m1532/Projects_MVP/geospatial/enviorment/HeatwaveData/DailyClimateData_CountyLevel/ClimateData_Daily/ClimateData_Daily_" + str(year) + ".csv", dtype={'fips': 'object'}).compute().drop(columns={'Unnamed: 0'})
    pattern.append(temp_year)

In [5]:
temp = pd.concat(pattern)
temp.head()

,fips,date,year,mean_temp,min_temp,max_temp,dewpoint,sea_level_pressure,station_pressure,visibility,...,precipitation,fog,rain,snow,hail,thunder,tornado,Relative_Humidity,Heat_Index,Month
0,01001,1960-01-01,1960,43.282353,40.029412,46.052941,33.658824,1020.929412,14.505882,7.870588,...,NaN,0.058824,1.0,0.0,0.0,0.000000,0.0,68.631040,40.536247,Jan
1,01001,1960-01-02,1960,42.764706,41.000000,45.176471,40.247059,1019.629412,13.205882,4.135294,...,NaN,1.000000,1.0,0.0,0.0,0.058824,0.0,90.742445,41.006071,Jan
2,01001,1960-01-03,1960,48.941176,44.317647,54.829412,42.147059,1016.994118,10.564706,9.094118,...,NaN,1.000000,1.0,0.0,0.0,0.000000,0.0,77.297475,47.168275,Jan
3,01001,1960-01-04,1960,42.117647,33.841176,52.917647,29.505882,1024.129412,17.617647,9.923529,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,60.689027,38.881796,Jan
4,01001,1960-01-05,1960,45.747059,41.194118,50.776471,31.423529,1023.488235,17.029412,8.888235,...,NaN,0.058824,1.0,0.0,0.0,0.000000,0.0,57.114836,42.706162,Jan


In [6]:
temp['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020])

In [7]:
import math

def water_vapor_pressure(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    return e


def apparent_temperature(temperature, water_vapor_pressure):
    A = -1.3 + 0.92 * temperature + 2.2 * water_vapor_pressure
    return A


temperature = 28 
water_vapor_pressure = water_vapor_pressure(temperature) 
apparent_temp = apparent_temperature(temperature, water_vapor_pressure)
print(f"Apparent temperature at {temperature} degrees Celsius and {water_vapor_pressure:.2f} kPa water vapor pressure is {apparent_temp:.2f} °C")

Apparent temperature at 28 degrees Celsius and 3.77 kPa water vapor pressure is 32.76 °C


In [8]:
def findApparentTemp(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    A = -1.3 + 0.92 * temperature + 2.2 * e
    return A

In [9]:
climate = temp[['year', 'Month', 'date', 'fips', 'mean_temp', 'min_temp', 'max_temp']]
climate = climate.rename(columns = {'Month': 'month'})
climate.head()

,year,month,date,fips,mean_temp,min_temp,max_temp
0,1960,Jan,1960-01-01,01001,43.282353,40.029412,46.052941
1,1960,Jan,1960-01-02,01001,42.764706,41.000000,45.176471
2,1960,Jan,1960-01-03,01001,48.941176,44.317647,54.829412
3,1960,Jan,1960-01-04,01001,42.117647,33.841176,52.917647
4,1960,Jan,1960-01-05,01001,45.747059,41.194118,50.776471


In [10]:
def fahrenheit_to_celsius(f):
    c = (f -32)*5/9
    return c

In [11]:
climate['mean_temp'] = climate['mean_temp'].apply(fahrenheit_to_celsius)
climate['min_temp'] = climate['min_temp'].apply(fahrenheit_to_celsius)
climate['max_temp'] = climate['max_temp'].apply(fahrenheit_to_celsius)
climate.head()

,year,month,date,fips,mean_temp,min_temp,max_temp
0,1960,Jan,1960-01-01,01001,6.267974,4.460784,7.807190
1,1960,Jan,1960-01-02,01001,5.980392,5.000000,7.320261
2,1960,Jan,1960-01-03,01001,9.411765,6.843137,12.683007
3,1960,Jan,1960-01-04,01001,5.620915,1.022876,11.620915
4,1960,Jan,1960-01-05,01001,7.637255,5.107843,10.431373


In [12]:
climate['AT_mean'] = climate['mean_temp'].apply(findApparentTemp)
climate['AT_min'] = climate['min_temp'].apply(findApparentTemp)
climate['AT_max'] = climate['max_temp'].apply(findApparentTemp)
climate.head()

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,Jan,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844
1,1960,Jan,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880
2,1960,Jan,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849
3,1960,Jan,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350
4,1960,Jan,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946


In [13]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}

In [14]:
climate['month'] = climate['month'].replace(month_replace)
climate.head()

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946


In [15]:
arizona = climate[(climate['fips'] == '04013') & (climate['month'] == 7)]
arizona

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
35852,1960,7,1960-07-01,04013,33.941251,24.381676,42.927321,41.577817,27.834627,57.136179
35853,1960,7,1960-07-02,04013,35.997743,28.804977,43.193866,44.876487,33.900068,57.647488
35854,1960,7,1960-07-03,04013,35.539423,30.353926,42.148882,44.129157,36.137477,55.661117
35855,1960,7,1960-07-04,04013,31.815934,24.988548,39.617498,38.309232,28.640535,51.043578
35856,1960,7,1960-07-05,04013,33.259993,24.801509,39.745898,40.515118,28.391314,51.271460
...,...,...,...,...,...,...,...,...,...,...
37845,2020,7,2020-07-27,04013,37.181147,29.055060,43.049609,46.839852,34.257052,57.370363
37846,2020,7,2020-07-28,04013,38.224315,30.589583,44.651470,48.612332,36.483503,60.501608
37847,2020,7,2020-07-29,04013,38.432111,30.297159,45.489704,48.970234,36.054350,62.188832
37848,2020,7,2020-07-30,04013,39.034853,30.173604,47.071798,50.017675,35.873726,65.469120


### After data imputation, 97% of the counties have daily apparent tempeature data from 1960 to 2020

In [16]:
fips_numbers = []
for year in years:
    dict_year = {}
    df_year = climate[climate['year'] == year]
    df_new = df_year.dropna(subset = 'AT_mean')
    numbers = len(df_new['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [17]:
fips_numbers

[{1960: 3027},
 {1961: 3020},
 {1962: 3017},
 {1963: 3016},
 {1964: 3019},
 {1965: 3021},
 {1966: 2964},
 {1967: 2965},
 {1968: 2965},
 {1969: 2955},
 {1970: 2948},
 {1971: 2951},
 {1972: 2887},
 {1973: 3210},
 {1974: 3205},
 {1975: 3211},
 {1976: 3220},
 {1977: 3218},
 {1978: 3220},
 {1979: 3221},
 {1980: 3221},
 {1981: 3222},
 {1982: 3222},
 {1983: 3223},
 {1984: 3223},
 {1985: 3223},
 {1986: 3223},
 {1987: 3224},
 {1988: 3224},
 {1989: 3224},
 {1990: 3224},
 {1991: 3225},
 {1992: 3225},
 {1993: 3225},
 {1994: 3226},
 {1995: 3226},
 {1996: 3223},
 {1997: 3219},
 {1998: 3226},
 {1999: 3227},
 {2000: 3227},
 {2001: 3227},
 {2002: 3226},
 {2003: 3227},
 {2004: 3227},
 {2005: 3226},
 {2006: 3226},
 {2007: 3227},
 {2008: 3227},
 {2009: 3227},
 {2010: 3227},
 {2011: 3227},
 {2012: 3228},
 {2013: 3228},
 {2014: 3228},
 {2015: 3227},
 {2016: 3227},
 {2017: 3228},
 {2018: 3227},
 {2019: 3227},
 {2020: 3227}]

In [18]:
climate['AT_mean'].describe()

count    6.865815e+07
mean     1.507401e+01
std      1.224596e+01
min     -5.434775e+01
25%      5.848152e+00
50%      1.605107e+01
75%      2.524169e+01
max      5.791631e+01
Name: AT_mean, dtype: float64

In [19]:
climate['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020])

In [20]:
climate = climate.sort_values(['fips', 'month', 'year'])
climate

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946
...,...,...,...,...,...,...,...,...,...,...
395662,2020,12,2020-12-27,78030,25.291667,23.097222,27.847222,29.046037,26.154223,32.547518
395663,2020,12,2020-12-28,78030,25.416667,23.347222,27.638889,29.213842,26.478660,32.256315
395664,2020,12,2020-12-29,78030,25.861111,23.069444,27.555556,29.813285,26.118252,32.140129
395665,2020,12,2020-12-30,78030,26.208333,25.027778,27.097222,30.284683,28.692903,31.504095


### find temperature data during 1981 to 2010

In [21]:
year_temp = [year for year in range(1981, 2011)]

In [22]:
temp = climate[climate['year'].isin(year_temp)]
temp

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1981,1,1981-01-01,01001,6.666667,-2.222222,16.722222,6.985022,-2.202125,18.264448
1,1981,1,1981-01-02,01001,6.888889,-0.611111,17.222222,7.222537,-0.576562,18.858992
2,1981,1,1981-01-03,01001,4.555556,-3.388889,16.722222,4.750014,-3.370214,18.264448
3,1981,1,1981-01-04,01001,5.666667,-3.277778,16.722222,5.921613,-3.259279,18.264448
4,1981,1,1981-01-05,01001,1.464052,-4.101307,6.640523,1.540599,-4.080006,6.957108
...,...,...,...,...,...,...,...,...,...,...
385725,2010,12,2010-12-27,78030,26.347222,24.527778,29.152778,30.474010,28.027930,34.396978
385726,2010,12,2010-12-28,78030,25.319444,23.875000,27.597222,29.083297,27.167723,32.198201
385727,2010,12,2010-12-29,78030,24.347222,23.583333,25.305556,27.789108,26.786225,29.064665
385728,2010,12,2010-12-30,78030,24.472222,22.694444,26.861111,27.954373,25.634127,31.178391


In [23]:
temp['mean_temp'].describe()

count    3.421093e+07
mean     1.349289e+01
std      1.076223e+01
min     -5.766667e+01
25%      5.756705e+00
50%      1.483333e+01
75%      2.242929e+01
max      4.322222e+01
Name: mean_temp, dtype: float64

In [24]:
fips_numbers = []
for year in year_temp:
    dict_year = {}
    df_year = temp[temp['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [25]:
fips_numbers

[{1981: 3222},
 {1982: 3222},
 {1983: 3223},
 {1984: 3223},
 {1985: 3223},
 {1986: 3223},
 {1987: 3224},
 {1988: 3224},
 {1989: 3224},
 {1990: 3224},
 {1991: 3225},
 {1992: 3225},
 {1993: 3225},
 {1994: 3226},
 {1995: 3226},
 {1996: 3223},
 {1997: 3219},
 {1998: 3226},
 {1999: 3227},
 {2000: 3227},
 {2001: 3227},
 {2002: 3226},
 {2003: 3227},
 {2004: 3227},
 {2005: 3226},
 {2006: 3226},
 {2007: 3227},
 {2008: 3227},
 {2009: 3227},
 {2010: 3227}]

In [26]:
import datetime

temp['date'] = pd.to_datetime(temp['date'])
temp.head()

/tmp/ipykernel_1071245/108427414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['date'] = pd.to_datetime(temp['date'])


,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1981,1,1981-01-01,01001,6.666667,-2.222222,16.722222,6.985022,-2.202125,18.264448
1,1981,1,1981-01-02,01001,6.888889,-0.611111,17.222222,7.222537,-0.576562,18.858992
2,1981,1,1981-01-03,01001,4.555556,-3.388889,16.722222,4.750014,-3.370214,18.264448
3,1981,1,1981-01-04,01001,5.666667,-3.277778,16.722222,5.921613,-3.259279,18.264448
4,1981,1,1981-01-05,01001,1.464052,-4.101307,6.640523,1.540599,-4.080006,6.957108


In [27]:
temp_valid = temp[(temp['month'] == 7) | (temp['month'] == 8)]
temp_valid

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
181,1981,7,1981-07-01,01001,24.771242,21.960784,31.366013,28.351055,24.694869,37.634468
182,1981,7,1981-07-02,01001,21.045752,19.591503,23.611111,23.537614,21.729355,26.822483
183,1981,7,1981-07-03,01001,23.526144,21.111111,26.846405,26.711625,23.619764,31.158148
184,1981,7,1981-07-04,01001,23.705882,20.000000,27.679739,26.946306,22.233569,32.313330
185,1981,7,1981-07-05,01001,25.349673,21.960784,28.683007,29.123865,24.694869,33.726538
...,...,...,...,...,...,...,...,...,...,...
385603,2010,8,2010-08-27,78030,29.180556,26.763889,32.402778,34.436800,31.044658,39.198174
385604,2010,8,2010-08-28,78030,29.333333,26.722222,32.097222,34.656175,30.987412,38.734063
385605,2010,8,2010-08-29,78030,30.083333,28.430556,32.111111,35.742020,33.368571,38.755099
385606,2010,8,2010-08-30,78030,28.277778,25.444444,29.916667,33.152716,29.251179,35.499428


In [28]:
temp_valid['mean_temp'].isna().sum()

0

In [29]:
temp_valid['max_temp'].isna().sum()

907

In [30]:
temp_valid['min_temp'].isna().sum()

2408

In [31]:
min_AT_p85 = temp_valid.groupby(['fips'])['AT_min'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
min_AT_p85.columns = ['fips', 'AT_min_p85']
min_AT_p85

,fips,AT_min_p85
0,01001,28.220786
1,01003,29.175568
2,01005,26.420490
3,01007,26.208209
4,01009,25.741472
...,...,...
3225,72151,30.725023
3226,72153,31.264655
3227,78010,32.372669
3228,78020,31.677060


In [31]:
mean_temp_p85 = temp_valid.groupby(['fips'])['mean_temp'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
mean_temp_p85.columns = ['fips', 'mean_temp_p85']
mean_temp_p85

,fips,mean_temp_p85
0,01001,30.095261
1,01003,29.283978
2,01005,28.537963
3,01007,28.567708
4,01009,28.056076
...,...,...
3225,72151,28.969136
3226,72153,30.480303
3227,78010,30.000000
3228,78020,30.277778


In [41]:
mean_AT_p85 = temp_valid.groupby(['fips'])['AT_mean'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
mean_AT_p85.columns = ['fips', 'AT_mean_p85']
mean_AT_p85

,fips,AT_mean_p85
0,01001,35.759410
1,01003,34.585238
2,01005,33.520675
3,01007,33.562850
4,01009,32.840515
...,...,...
3225,72151,34.134217
3226,72153,36.322853
3227,78010,35.620631
3228,78020,36.025990


In [32]:
min_temp_p85 = temp_valid.groupby(['fips'])['min_temp'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
min_temp_p85.columns = ['fips', 'min_temp_p85']
min_temp_p85

,fips,min_temp_p85
0,01001,24.673203
1,01003,25.388178
2,01005,23.302469
3,01007,23.138889
4,01009,22.777778
...,...,...
3225,72151,26.530864
3226,72153,26.923737
3227,78010,27.722222
3228,78020,27.222222


In [33]:
max_temp_p85 = temp_valid.groupby(['fips'])['max_temp'].apply(lambda x: np.percentile(x.dropna(), 85)).reset_index()
max_temp_p85.columns = ['fips', 'max_temp_p85']
max_temp_p85

,fips,max_temp_p85
0,01001,35.997222
1,01003,33.505168
2,01005,35.429762
3,01007,35.336806
4,01009,35.078472
...,...,...
3225,72151,32.456790
3226,72153,33.181818
3227,78010,32.222222
3228,78020,33.777778


In [34]:
max_temp_p85['max_temp_p85'].isna().sum()

0

In [42]:
temp_p85 = mean_temp_p85.merge(mean_AT_p85, on = ['fips'], how = 'inner')
temp_p85.head()

,fips,mean_temp_p85,AT_mean_p85
0,01001,30.095261,35.759410
1,01003,29.283978,34.585238
2,01005,28.537963,33.520675
3,01007,28.567708,33.562850
4,01009,28.056076,32.840515


In [43]:
temp_p85

,fips,mean_temp_p85,AT_mean_p85
0,01001,30.095261,35.759410
1,01003,29.283978,34.585238
2,01005,28.537963,33.520675
3,01007,28.567708,33.562850
4,01009,28.056076,32.840515
...,...,...,...
3225,72151,28.969136,34.134217
3226,72153,30.480303,36.322853
3227,78010,30.000000,35.620631
3228,78020,30.277778,36.025990


In [44]:
climate

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946
...,...,...,...,...,...,...,...,...,...,...
395662,2020,12,2020-12-27,78030,25.291667,23.097222,27.847222,29.046037,26.154223,32.547518
395663,2020,12,2020-12-28,78030,25.416667,23.347222,27.638889,29.213842,26.478660,32.256315
395664,2020,12,2020-12-29,78030,25.861111,23.069444,27.555556,29.813285,26.118252,32.140129
395665,2020,12,2020-12-30,78030,26.208333,25.027778,27.097222,30.284683,28.692903,31.504095


### Find the heatwave Count

In [45]:
def findList(fips, mean_temp_p85):
    temp_p85_list = []
    temp_p85_list.append(fips)
    temp_p85_list.append(mean_temp_p85)
    return temp_p85_list

In [47]:
def findATList(fips, mean_AT_p85):
    temp_p85_list = []
    temp_p85_list.append(fips)
    temp_p85_list.append(mean_AT_p85)
    return temp_p85_list

In [48]:
temp_p85['temp_p85_list'] = temp_p85.apply(lambda x: findList(x['fips'], x['mean_temp_p85']), axis=1)
temp_p85.head()

,fips,mean_temp_p85,AT_mean_p85,temp_p85_list
0,01001,30.095261,35.759410,"[01001, 30.095261437908505]"
1,01003,29.283978,34.585238,"[01003, 29.28397767176838]"
2,01005,28.537963,33.520675,"[01005, 28.537962962962965]"
3,01007,28.567708,33.562850,"[01007, 28.567708333333332]"
4,01009,28.056076,32.840515,"[01009, 28.056076388888894]"


In [50]:
temp_p85['AT_p85_list'] = temp_p85.apply(lambda x: findList(x['fips'], x['AT_mean_p85']), axis=1)
temp_p85.head()

,fips,mean_temp_p85,AT_mean_p85,temp_p85_list,AT_p85_list
0,01001,30.095261,35.759410,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
1,01003,29.283978,34.585238,"[01003, 29.28397767176838]","[01003, 34.58523817888881]"
2,01005,28.537963,33.520675,"[01005, 28.537962962962965]","[01005, 33.52067492945842]"
3,01007,28.567708,33.562850,"[01007, 28.567708333333332]","[01007, 33.56284987331675]"
4,01009,28.056076,32.840515,"[01009, 28.056076388888894]","[01009, 32.84051545008569]"


In [51]:
temp_p85_list = temp_p85['temp_p85_list'].tolist()
temp_p85_list[:5]

[['01001', 30.095261437908505],
 ['01003', 29.28397767176838],
 ['01005', 28.537962962962965],
 ['01007', 28.567708333333332],
 ['01009', 28.056076388888894]]

In [52]:
AT_p85_list = temp_p85['AT_p85_list'].tolist()
AT_p85_list[:5]

[['01001', 35.75941029796386],
 ['01003', 34.58523817888881],
 ['01005', 33.52067492945842],
 ['01007', 33.56284987331675],
 ['01009', 32.84051545008569]]

In [53]:
def findHeatwaveCount(temp, fips, temp_p85):
    df = temp[temp['fips'] == fips]
    c1 = df['AT_min'].ge(temp_p85)
    N = 2
    g = (c1 != c1.shift()).cumsum()
    df1 = df.assign(
        cnt=df.groupby(g).date.transform('count')
      , n=df.groupby(g).agg('cumcount')
      , g=g
    )
    c2 = df1.cnt.ge(N)
    c3 = df1.n.mod(N).eq(0) & df1.n.le(df1.cnt-N)
    df['flag'] = np.where(c1 & c2 & c3, 1, 0)
    del(df1)
    return df

In [54]:
climate

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946
...,...,...,...,...,...,...,...,...,...,...
395662,2020,12,2020-12-27,78030,25.291667,23.097222,27.847222,29.046037,26.154223,32.547518
395663,2020,12,2020-12-28,78030,25.416667,23.347222,27.638889,29.213842,26.478660,32.256315
395664,2020,12,2020-12-29,78030,25.861111,23.069444,27.555556,29.813285,26.118252,32.140129
395665,2020,12,2020-12-30,78030,26.208333,25.027778,27.097222,30.284683,28.692903,31.504095


In [33]:
def findDecade(year):
    if year >= 1960 and year <= 1969:
        return '1960s'
    elif year >= 1970 and year <= 1979:
        return '1970s'
    elif year >= 1980 and year <= 1989:
        return '1980s'
    elif year >= 1990 and year <= 1999:
        return '1990s'
    elif year >= 2000 and year <= 2009:
        return '2000s'
    elif year >= 2010 and year <= 2019:
        return '2010s'
    else:
        return '2020s'

In [34]:
climate['decade'] = climate['year'].apply(findDecade)
climate

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max,decade
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844,1960s
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880,1960s
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849,1960s
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350,1960s
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946,1960s
...,...,...,...,...,...,...,...,...,...,...,...
395662,2020,12,2020-12-27,78030,25.291667,23.097222,27.847222,29.046037,26.154223,32.547518,2020s
395663,2020,12,2020-12-28,78030,25.416667,23.347222,27.638889,29.213842,26.478660,32.256315,2020s
395664,2020,12,2020-12-29,78030,25.861111,23.069444,27.555556,29.813285,26.118252,32.140129,2020s
395665,2020,12,2020-12-30,78030,26.208333,25.027778,27.097222,30.284683,28.692903,31.504095,2020s


In [35]:
climate_1960s = climate[climate['decade'] == '1960s']
climate_1970s = climate[climate['decade'] == '1970s']
climate_1980s = climate[climate['decade'] == '1980s']
climate_1990s = climate[climate['decade'] == '1990s']
climate_2000s = climate[climate['decade'] == '2000s']
climate_2010s = climate[climate['decade'] == '2010s']
climate_2020s = climate[climate['decade'] == '2020s']

In [37]:
df_1960s = climate_1960s.merge(min_AT_p85, on='fips', how='left')

In [38]:
df_1960s

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max,decade,AT_min_p85
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844,1960s,28.220786
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880,1960s,28.220786
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849,1960s,28.220786
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350,1960s,28.220786
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946,1960s,28.220786
...,...,...,...,...,...,...,...,...,...,...,...,...
10735699,1969,12,1969-12-27,78030,25.111111,21.111111,30.611111,28.804255,23.619764,36.515190,1960s,31.677060
10735700,1969,12,1969-12-28,78030,26.388889,23.888889,29.388889,30.530895,27.185933,34.736099,1960s,31.677060
10735701,1969,12,1969-12-29,78030,25.777778,22.222222,28.888889,29.700554,25.028388,34.019670,1960s,31.677060
10735702,1969,12,1969-12-30,78030,27.333333,26.111111,29.388889,31.831120,30.152416,34.736099,1960s,31.677060


In [39]:
df_1970s = climate_1970s.merge(min_AT_p85, on='fips', how='left')
df_1980s = climate_1980s.merge(min_AT_p85, on='fips', how='left')
df_1990s = climate_1990s.merge(min_AT_p85, on='fips', how='left')
df_2000s = climate_2000s.merge(min_AT_p85, on='fips', how='left')
df_2010s = climate_2010s.merge(min_AT_p85, on='fips', how='left')
df_2020s = climate_2020s.merge(min_AT_p85, on='fips', how='left')

In [40]:
# Function to count heatwaves
def count_heatwaves(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Count the groups that are at least two days long
    heatwave_count = (group_lengths >= 2).sum()

    return heatwave_count

In [63]:
heatwave_counts_1960s_temp = df_1960s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_1960s_temp = heatwave_counts_1960s_temp.reset_index(name='heatwave_count')
heatwave_counts_1960s_temp

,year,month,fips,heatwave_count
0,1960,1,01001,0
1,1960,1,01003,0
2,1960,1,01005,0
3,1960,1,01007,0
4,1960,1,01009,0
...,...,...,...,...
354639,1969,12,72151,1
354640,1969,12,72153,0
354641,1969,12,78010,1
354642,1969,12,78020,0


In [67]:
heatwave_counts_1960s_temp['heatwave_count'].describe()

count    354644.000000
mean          0.047380
std           0.313948
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [64]:
df_1960s

,year,month,date,fips,mean_temp,min_temp,max_temp,AT_mean,AT_min,AT_max,decade,mean_temp_p85,AT_mean_p85,temp_p85_list,AT_p85_list
0,1960,1,1960-01-01,01001,6.267974,4.460784,7.807190,6.559999,4.650550,8.208844,1960s,30.095261,35.75941,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
1,1960,1,1960-01-02,01001,5.980392,5.000000,7.320261,6.254294,5.217431,7.684880,1960s,30.095261,35.75941,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
2,1960,1,1960-01-03,01001,9.411765,6.843137,12.683007,9.951747,7.173600,13.589849,1960s,30.095261,35.75941,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
3,1960,1,1960-01-04,01001,5.620915,1.022876,11.620915,5.873167,1.088155,12.395350,1960s,30.095261,35.75941,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
4,1960,1,1960-01-05,01001,7.637255,5.107843,10.431373,8.025731,5.331092,11.072946,1960s,30.095261,35.75941,"[01001, 30.095261437908505]","[01001, 35.75941029796386]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10735699,1969,12,1969-12-27,78030,25.111111,21.111111,30.611111,28.804255,23.619764,36.515190,1960s,30.277778,36.02599,"[78030, 30.27777777777778]","[78030, 36.025989766347365]"
10735700,1969,12,1969-12-28,78030,26.388889,23.888889,29.388889,30.530895,27.185933,34.736099,1960s,30.277778,36.02599,"[78030, 30.27777777777778]","[78030, 36.025989766347365]"
10735701,1969,12,1969-12-29,78030,25.777778,22.222222,28.888889,29.700554,25.028388,34.019670,1960s,30.277778,36.02599,"[78030, 30.27777777777778]","[78030, 36.025989766347365]"
10735702,1969,12,1969-12-30,78030,27.333333,26.111111,29.388889,31.831120,30.152416,34.736099,1960s,30.277778,36.02599,"[78030, 30.27777777777778]","[78030, 36.025989766347365]"


In [41]:
heatwave_counts_1960s_AT = df_1960s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_1960s_AT = heatwave_counts_1960s_AT.reset_index(name='heatwave_count')
heatwave_counts_1960s_AT

,year,month,fips,heatwave_count
0,1960,1,01001,0
1,1960,1,01003,0
2,1960,1,01005,0
3,1960,1,01007,0
4,1960,1,01009,0
...,...,...,...,...
354639,1969,12,72151,0
354640,1969,12,72153,0
354641,1969,12,78010,0
354642,1969,12,78020,0


In [42]:
heatwave_counts_1960s_AT['heatwave_count'].describe()

count    354644.000000
mean          0.224349
std           0.680197
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [68]:
heatwave_counts_1970s_temp = df_1970s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_1970s_temp = heatwave_counts_1970s_temp.reset_index(name='heatwave_count')
heatwave_counts_1970s_temp

,year,month,fips,heatwave_count
0,1970,1,01001,0
1,1970,1,01003,0
2,1970,1,01005,0
3,1970,1,01007,0
4,1970,1,01009,0
...,...,...,...,...
365959,1979,12,72151,0
365960,1979,12,72153,0
365961,1979,12,78010,0
365962,1979,12,78020,0


In [69]:
heatwave_counts_1970s_temp['heatwave_count'].describe()

count    365964.000000
mean          0.059683
std           0.377176
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [43]:
heatwave_counts_1970s_AT = df_1970s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_1970s_AT = heatwave_counts_1970s_AT.reset_index(name='heatwave_count')
heatwave_counts_1970s_AT

,year,month,fips,heatwave_count
0,1970,1,01001,0
1,1970,1,01003,0
2,1970,1,01005,0
3,1970,1,01007,0
4,1970,1,01009,0
...,...,...,...,...
365959,1979,12,72151,0
365960,1979,12,72153,0
365961,1979,12,78010,0
365962,1979,12,78020,0


In [44]:
heatwave_counts_1970s_AT['heatwave_count'].describe()

count    365964.000000
mean          0.200514
std           0.621450
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [75]:
heatwave_counts_1980s_temp = df_1980s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_1980s_temp = heatwave_counts_1980s_temp.reset_index(name='heatwave_count')
heatwave_counts_1980s_temp

,year,month,fips,heatwave_count
0,1980,1,01001,0
1,1980,1,01003,0
2,1980,1,01005,0
3,1980,1,01007,0
4,1980,1,01009,0
...,...,...,...,...
378630,1989,12,72149,0
378631,1989,12,72151,0
378632,1989,12,72153,0
378633,1989,12,78020,0


In [77]:
heatwave_counts_1980s_temp['heatwave_count'].describe()

count    378635.000000
mean          0.069349
std           0.392884
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [45]:
heatwave_counts_1980s_AT = df_1980s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_1980s_AT = heatwave_counts_1980s_AT.reset_index(name='heatwave_count')
heatwave_counts_1980s_AT

,year,month,fips,heatwave_count
0,1980,1,01001,0
1,1980,1,01003,0
2,1980,1,01005,0
3,1980,1,01007,0
4,1980,1,01009,0
...,...,...,...,...
378630,1989,12,72149,0
378631,1989,12,72151,0
378632,1989,12,72153,0
378633,1989,12,78020,0


In [46]:
heatwave_counts_1980s_AT['heatwave_count'].describe()

count    378635.000000
mean          0.230444
std           0.668342
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [80]:
heatwave_counts_1990s_temp = df_1990s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_1990s_temp = heatwave_counts_1990s_temp.reset_index(name='heatwave_count')
heatwave_counts_1990s_temp

,year,month,fips,heatwave_count
0,1990,1,01001,0
1,1990,1,01003,0
2,1990,1,01005,0
3,1990,1,01007,0
4,1990,1,01009,0
...,...,...,...,...
380856,1999,12,72151,0
380857,1999,12,72153,0
380858,1999,12,78010,0
380859,1999,12,78020,0


In [81]:
heatwave_counts_1990s_temp['heatwave_count'].describe()

count    380861.000000
mean          0.058105
std           0.357910
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [47]:
heatwave_counts_1990s_AT = df_1990s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_1990s_AT = heatwave_counts_1990s_AT.reset_index(name='heatwave_count')
heatwave_counts_1990s_AT

,year,month,fips,heatwave_count
0,1990,1,01001,0
1,1990,1,01003,0
2,1990,1,01005,0
3,1990,1,01007,0
4,1990,1,01009,0
...,...,...,...,...
380856,1999,12,72151,0
380857,1999,12,72153,0
380858,1999,12,78010,0
380859,1999,12,78020,0


In [48]:
heatwave_counts_1990s_AT['heatwave_count'].describe()

count    380861.000000
mean          0.208380
std           0.626596
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [84]:
heatwave_counts_2000s_temp = df_2000s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_2000s_temp = heatwave_counts_2000s_temp.reset_index(name='heatwave_count')
heatwave_counts_2000s_temp

,year,month,fips,heatwave_count
0,2000,1,01001,0
1,2000,1,01003,0
2,2000,1,01005,0
3,2000,1,01007,0
4,2000,1,01009,0
...,...,...,...,...
382606,2009,12,72151,0
382607,2009,12,72153,0
382608,2009,12,78010,0
382609,2009,12,78020,0


In [85]:
heatwave_counts_2000s_temp['heatwave_count'].describe()

count    382611.000000
mean          0.046368
std           0.314825
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [49]:
heatwave_counts_2000s_AT = df_2000s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_2000s_AT = heatwave_counts_2000s_AT.reset_index(name='heatwave_count')
heatwave_counts_2000s_AT

,year,month,fips,heatwave_count
0,2000,1,01001,0
1,2000,1,01003,0
2,2000,1,01005,0
3,2000,1,01007,0
4,2000,1,01009,0
...,...,...,...,...
382606,2009,12,72151,0
382607,2009,12,72153,0
382608,2009,12,78010,0
382609,2009,12,78020,0


In [54]:
heatwave_counts_2000s_AT['heatwave_count'].describe()

count    382611.000000
mean          0.175539
std           0.562024
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [88]:
heatwave_counts_2010s_temp = df_2010s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_2010s_temp = heatwave_counts_2010s_temp.reset_index(name='heatwave_count')
heatwave_counts_2010s_temp

,year,month,fips,heatwave_count
0,2010,1,01001,0
1,2010,1,01003,0
2,2010,1,01005,0
3,2010,1,01007,0
4,2010,1,01009,0
...,...,...,...,...
383178,2019,12,72151,1
383179,2019,12,72153,0
383180,2019,12,78010,1
383181,2019,12,78020,3


In [89]:
heatwave_counts_2010s_temp['heatwave_count'].describe()

count    383183.000000
mean          0.071645
std           0.412934
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           7.000000
Name: heatwave_count, dtype: float64

In [51]:
heatwave_counts_2010s_AT = df_2010s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_2010s_AT = heatwave_counts_2010s_AT.reset_index(name='heatwave_count')
heatwave_counts_2010s_AT

,year,month,fips,heatwave_count
0,2010,1,01001,0
1,2010,1,01003,0
2,2010,1,01005,0
3,2010,1,01007,0
4,2010,1,01009,0
...,...,...,...,...
383178,2019,12,72151,0
383179,2019,12,72153,0
383180,2019,12,78010,0
383181,2019,12,78020,0


In [91]:
heatwave_counts_2010s_AT['heatwave_count'].describe()

count    383183.000000
mean          0.001091
std           0.038076
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           4.000000
Name: heatwave_count, dtype: float64

In [92]:
heatwave_counts_2020s_temp = df_2020s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['mean_temp_p85'].iloc[0]))
heatwave_counts_2020s_temp = heatwave_counts_2020s_temp.reset_index(name='heatwave_count')
heatwave_counts_2020s_temp

,year,month,fips,heatwave_count
0,2020,1,01001,0
1,2020,1,01003,0
2,2020,1,01005,0
3,2020,1,01007,0
4,2020,1,01009,0
...,...,...,...,...
38453,2020,12,72149,0
38454,2020,12,72153,0
38455,2020,12,78010,0
38456,2020,12,78020,0


In [93]:
heatwave_counts_2020s_temp['heatwave_count'].describe()

count    38458.000000
mean         0.066696
std          0.398398
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          6.000000
Name: heatwave_count, dtype: float64

In [52]:
heatwave_counts_2020s_AT = df_2020s.groupby(['year', 'month', 'fips']).apply(lambda group: count_heatwaves(group['AT_min'], group['AT_min_p85'].iloc[0]))
heatwave_counts_2020s_AT = heatwave_counts_2020s_AT.reset_index(name='heatwave_count')
heatwave_counts_2020s_AT

,year,month,fips,heatwave_count
0,2020,1,01001,0
1,2020,1,01003,0
2,2020,1,01005,0
3,2020,1,01007,0
4,2020,1,01009,0
...,...,...,...,...
38453,2020,12,72149,0
38454,2020,12,72153,0
38455,2020,12,78010,0
38456,2020,12,78020,0


In [53]:
heatwave_counts_2020s_AT['heatwave_count'].describe()

count    38458.000000
mean         0.255187
std          0.694998
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          6.000000
Name: heatwave_count, dtype: float64

In [55]:
count_AT = [heatwave_counts_1960s_AT,heatwave_counts_1970s_AT, heatwave_counts_1980s_AT, heatwave_counts_1990s_AT, heatwave_counts_2000s_AT,
              heatwave_counts_2010s_AT,heatwave_counts_2020s_AT]

In [96]:
count_temp = [heatwave_counts_1960s_temp,heatwave_counts_1970s_temp, heatwave_counts_1980s_temp, heatwave_counts_1990s_temp, heatwave_counts_2000s_temp,
              heatwave_counts_2010s_temp,heatwave_counts_2020s_temp]
count_AT = [heatwave_counts_1960s_AT,heatwave_counts_1970s_AT, heatwave_counts_1980s_AT, heatwave_counts_1990s_AT, heatwave_counts_2000s_AT,
              heatwave_counts_2010s_AT,heatwave_counts_2020s_AT]

In [97]:
heatwave_count_temp = pd.concat(count_temp)
heatwave_count_temp

,year,month,fips,heatwave_count
0,1960,1,01001,0
1,1960,1,01003,0
2,1960,1,01005,0
3,1960,1,01007,0
4,1960,1,01009,0
...,...,...,...,...
38453,2020,12,72149,0
38454,2020,12,72153,0
38455,2020,12,78010,0
38456,2020,12,78020,0


In [56]:
heatwave_count_AT = pd.concat(count_AT)
heatwave_count_AT

,year,month,fips,heatwave_count
0,1960,1,01001,0
1,1960,1,01003,0
2,1960,1,01005,0
3,1960,1,01007,0
4,1960,1,01009,0
...,...,...,...,...
38453,2020,12,72149,0
38454,2020,12,72153,0
38455,2020,12,78010,0
38456,2020,12,78020,0


In [57]:
heatwave_count_AT.to_csv('heatwave_details/NOAA_min_AT_p85/heatwave_count_monthly_1960_2020.csv')

In [99]:
heatwave_count_temp.to_csv('heatwave_details/NOAA_mean_temp_p85/heatwave_count_monthly_1960_2020.csv')

In [100]:
heatwave_count_AT.to_csv('heatwave_details/NOAA_mean_AT_p85/heatwave_count_monthly_1960_2020.csv')

In [101]:
heatwave_count_temp['heatwave_count'].unique()

array([0, 1, 4, 3, 2, 6, 5, 7])

In [102]:
heatwave_count_AT['heatwave_count'].unique()

array([0, 1, 3, 2, 5, 4, 6])

### calculating heatwave_durations

In [104]:
df_data = [df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s]

In [ ]:
df = pd.concat(df_data)
df

In [63]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and has columns 'city', 'date', 'temperature'
# Assuming percentile_dict is your dictionary with 85th percentile temperatures for each city

# Sort by city and date
df = df.sort_values(['fips', 'month', 'date'])

In [64]:
years = [year for year in range(1960, 2021)]

In [65]:
# Define a function to calculate heatwave durations for each city
def calculate_heatwave(df, years):
    pattern = []
    for year in years:
        # Create a boolean series where True indicates a day is part of a heatwave
        city_df = df[df['year'] == year] 
        for fips in city_df['fips'].unique():
            df_fips = city_df[city_df['fips'] == fips]
            percentile = df_fips['mean_temp_p85']
            is_heatwave = df_fips['AT_min'] > percentile

            # Identify where True values change to False and vice versa,
            # and assign each group of consecutive True values a unique identifier
            heatwave_id = (is_heatwave.diff() != 0).cumsum()

            # Group by the identifiers and count the size of each group
            heatwave_lengths = heatwave_id[is_heatwave].value_counts()

            # Filter out heatwaves that are less than 2 days long
            heatwave_lengths = heatwave_lengths[heatwave_lengths >= 2]

            # Create a DataFrame with the start date of each heatwave and its duration
            heatwaves = pd.DataFrame({
                'fips': fips,
                'duration': heatwave_lengths.values,
                'year': year
            })
            pattern.append(heatwaves)
    
    return pattern

In [66]:
pattern = calculate_heatwave(df, years)
heatwave_durations = pd.concat(pattern)
heatwave_durations

,fips,duration,year
0,01045,2,1960
0,01047,5,1960
1,01047,3,1960
2,01047,2,1960
3,01047,2,1960
...,...,...,...
0,55025,2,2020
0,55029,2,2020
0,55043,2,2020
0,55051,2,2020


In [67]:
# Function to calculate heatwave durations
def calculate_heatwave_durations(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Count the length of each group
    group_lengths = heatwave_groups[is_above_threshold].value_counts()

    # Filter for groups that are at least two days long, and return their lengths
    heatwave_durations = group_lengths[group_lengths >= 2]

    return heatwave_durations

In [68]:
# Apply the function to each year and fips
heatwave_durations = df.groupby(['year', 'fips']).apply(lambda group: calculate_heatwave_durations(group['AT_min'], group['mean_temp_p85'].iloc[0]))
# This will result in a series where each element is a series of heat wave durations
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_durations = heatwave_durations.explode().reset_index(name='heatwave_duration')
heatwave_durations

,year,fips,AT_min,heatwave_duration
0,1960,01045,6,2
1,1960,01047,4,5
2,1960,01047,18,3
3,1960,01047,8,2
4,1960,01047,10,2
...,...,...,...,...
41738,2020,55025,8,2
41739,2020,55029,2,2
41740,2020,55043,8,2
41741,2020,55051,4,2


In [69]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_durations[heatwave_durations['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [70]:
fips_numbers

[{1960: 65},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 221},
 {1969: 192},
 {1970: 148},
 {1971: 64},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 256},
 {1979: 155},
 {1980: 367},
 {1981: 278},
 {1982: 127},
 {1983: 367},
 {1984: 66},
 {1985: 97},
 {1986: 246},
 {1987: 286},
 {1988: 355},
 {1989: 172},
 {1990: 134},
 {1991: 203},
 {1992: 97},
 {1993: 261},
 {1994: 111},
 {1995: 413},
 {1996: 112},
 {1997: 249},
 {1998: 360},
 {1999: 709},
 {2000: 217},
 {2001: 468},
 {2002: 458},
 {2003: 248},
 {2004: 197},
 {2005: 677},
 {2006: 647},
 {2007: 614},
 {2008: 201},
 {2009: 287},
 {2010: 843},
 {2011: 1017},
 {2012: 542},
 {2013: 531},
 {2014: 196},
 {2015: 453},
 {2016: 756},
 {2017: 417},
 {2018: 656},
 {2019: 743},
 {2020: 504}]

In [71]:
heatwave_durations.to_csv('heatwave_1960_2020/heatwave_duration_1960_2020.csv')

In [88]:
df['date'] = pd.to_datetime(df['date'])

In [89]:
# Function to calculate heatwave dates
def calculate_heatwave_dates(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()
    
    # Get the dates of each group
    group_dates = series.loc[is_above_threshold, 'date'].groupby(heatwave_groups)

    # Filter for groups that are at least two days long, and return their dates
    heatwave_dates = group_dates.filter(lambda dates: len(dates) >= 2)

    return heatwave_dates

In [ ]:
# Apply the function to each year and fips
heatwave_dates = df.groupby(['year', 'month', 'fips']).apply(lambda group: calculate_heatwave_dates(group[['date', 'AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_dates

In [75]:
# The season length for each year and fips is the difference between the max and min date
season_lengths = heatwave_dates.groupby(level=['year','month','fips']).apply(lambda dates: dates.max() - dates.min())
season_lengths = season_lengths.explode().reset_index(name='heatwave_season_length')
season_lengths

,year,fips,heatwave_season_length
0,1960,01045,1 days
1,1960,01047,41 days
2,1960,01061,1 days
3,1960,01113,21 days
4,1960,05045,25 days
...,...,...,...
18136,2020,55025,1 days
18137,2020,55029,1 days
18138,2020,55043,1 days
18139,2020,55051,1 days


In [76]:
def findDaysNumber(days):
    days = str(days)
    return int(days.split()[0])+1

In [77]:
season_lengths['heatwave_length'] = season_lengths['heatwave_season_length'].apply(findDaysNumber)
season_lengths

,year,fips,heatwave_season_length,heatwave_length
0,1960,01045,1 days,2
1,1960,01047,41 days,42
2,1960,01061,1 days,2
3,1960,01113,21 days,22
4,1960,05045,25 days,26
...,...,...,...,...
18136,2020,55025,1 days,2
18137,2020,55029,1 days,2
18138,2020,55043,1 days,2
18139,2020,55051,1 days,2


In [78]:
season_lengths = season_lengths.drop('heatwave_season_length', axis = 1)
season_lengths = season_lengths.rename(columns = {'heatwave_length': 'heatwave_season_length'})
season_lengths

,year,fips,heatwave_season_length
0,1960,01045,2
1,1960,01047,42
2,1960,01061,2
3,1960,01113,22
4,1960,05045,26
...,...,...,...
18136,2020,55025,2
18137,2020,55029,2
18138,2020,55043,2
18139,2020,55051,2


In [79]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = season_lengths[season_lengths['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [80]:
fips_numbers

[{1960: 65},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 221},
 {1969: 192},
 {1970: 148},
 {1971: 64},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 256},
 {1979: 155},
 {1980: 367},
 {1981: 278},
 {1982: 127},
 {1983: 367},
 {1984: 66},
 {1985: 97},
 {1986: 246},
 {1987: 286},
 {1988: 355},
 {1989: 172},
 {1990: 134},
 {1991: 203},
 {1992: 97},
 {1993: 261},
 {1994: 111},
 {1995: 413},
 {1996: 112},
 {1997: 249},
 {1998: 360},
 {1999: 709},
 {2000: 217},
 {2001: 468},
 {2002: 458},
 {2003: 248},
 {2004: 197},
 {2005: 677},
 {2006: 647},
 {2007: 614},
 {2008: 201},
 {2009: 287},
 {2010: 843},
 {2011: 1017},
 {2012: 542},
 {2013: 531},
 {2014: 196},
 {2015: 453},
 {2016: 756},
 {2017: 417},
 {2018: 656},
 {2019: 743},
 {2020: 504}]

In [81]:
season_lengths.to_csv('heatwave_1960_2020/heatwave_season_length_1960_2020.csv')

In [82]:
def celsius_to_fahrenheit(c):
    f = c * 9/5 + 32
    return f

In [83]:
df['AT_min'] = df['AT_min'].apply(celsius_to_fahrenheit)
df['mean_temp_p85'] = df['mean_temp_p85'].apply(celsius_to_fahrenheit)

In [75]:
# Function to calculate average exceeded temperatures above threshold during heatwaves
def calculate_heatwave_exceeded_temp_avgs(series, threshold):
    # Create a boolean series where True represents exceeding the threshold
    is_above_threshold = series['AT_min'] > threshold

    # Identify groups of consecutive days above threshold
    # The diff and cumsum methods identify groups of consecutive True values
    heatwave_groups = (is_above_threshold.diff(1) != 0).cumsum()

    # Get the mean of exceeded temperatures (temperature - threshold) above threshold in each group
    group_exceeded_temp_avgs = (series.loc[is_above_threshold, 'AT_min'] - threshold).groupby(heatwave_groups).mean()

    # Filter for groups that are at least two days long, and return their exceeded temperature averages
    heatwave_exceeded_temp_avgs = group_exceeded_temp_avgs[group_exceeded_temp_avgs.index.map(is_above_threshold.groupby(heatwave_groups).size() >= 2)]

    return heatwave_exceeded_temp_avgs

In [76]:
# Apply the function to each year and fips
heatwave_exceeded_temp_avgs = df.groupby(['year', 'fips']).apply(lambda group: calculate_heatwave_exceeded_temp_avgs(group[['AT_min']], group['mean_temp_p85'].iloc[0]))
heatwave_exceeded_temp_avgs 

year  fips   AT_min
1960  01045  6         1.126089
      01047  4         2.826390
             8         0.267139
             10        3.829449
             14        1.010165
                         ...   
2020  55025  8         0.621146
      55029  2         1.515584
      55043  8         0.777385
      55051  4         2.759159
      55061  2         1.067584
Name: AT_min, Length: 41836, dtype: float64

In [77]:
# This will result in a series where each element is a series of heat wave exceeded temperature averages
# To convert this into a DataFrame where each row represents a heat wave, you can do:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.explode().reset_index(name='heatwave_exceeded_temp_avg')
heatwave_exceeded_temp_avgs

,year,fips,AT_min,heatwave_exceeded_temp_avg
0,1960,01045,6,1.126089
1,1960,01047,4,2.826390
2,1960,01047,8,0.267139
3,1960,01047,10,3.829449
4,1960,01047,14,1.010165
...,...,...,...,...
41831,2020,55025,8,0.621146
41832,2020,55029,2,1.515584
41833,2020,55043,8,0.777385
41834,2020,55051,4,2.759159


In [78]:
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.drop('AT_min', axis = 1)
heatwave_exceeded_temp_avgs = heatwave_exceeded_temp_avgs.rename(columns = {'heatwave_exceeded_temp_avg': 'heatwave_intensity'})
heatwave_exceeded_temp_avgs

,year,fips,heatwave_intensity
0,1960,01045,1.126089
1,1960,01047,2.826390
2,1960,01047,0.267139
3,1960,01047,3.829449
4,1960,01047,1.010165
...,...,...,...
41831,2020,55025,0.621146
41832,2020,55029,1.515584
41833,2020,55043,0.777385
41834,2020,55051,2.759159


In [79]:
fips_numbers = []
for year in years_30:
    dict_year = {}
    df_year = heatwave_exceeded_temp_avgs[heatwave_exceeded_temp_avgs['year'] == year]
    numbers = len(df_year['fips'].unique())
    dict_year[year] = numbers
    fips_numbers.append(dict_year)

In [80]:
fips_numbers

[{1960: 67},
 {1961: 70},
 {1962: 57},
 {1963: 70},
 {1964: 96},
 {1965: 61},
 {1966: 170},
 {1967: 38},
 {1968: 222},
 {1969: 192},
 {1970: 148},
 {1971: 65},
 {1972: 118},
 {1973: 179},
 {1974: 53},
 {1975: 105},
 {1976: 96},
 {1977: 245},
 {1978: 263},
 {1979: 160},
 {1980: 376},
 {1981: 278},
 {1982: 128},
 {1983: 373},
 {1984: 66},
 {1985: 97},
 {1986: 250},
 {1987: 291},
 {1988: 358},
 {1989: 173},
 {1990: 134},
 {1991: 204},
 {1992: 97},
 {1993: 263},
 {1994: 111},
 {1995: 414},
 {1996: 115},
 {1997: 253},
 {1998: 360},
 {1999: 712},
 {2000: 217},
 {2001: 469},
 {2002: 468},
 {2003: 253},
 {2004: 206},
 {2005: 681},
 {2006: 658},
 {2007: 617},
 {2008: 204},
 {2009: 289},
 {2010: 845},
 {2011: 1017},
 {2012: 542},
 {2013: 534},
 {2014: 196},
 {2015: 453},
 {2016: 757},
 {2017: 419},
 {2018: 660},
 {2019: 748},
 {2020: 504}]

In [81]:
heatwave_exceeded_temp_avgs.to_csv('heatwave_1960_2020/heatwave_intensity_1960_2020.csv')